## Build USHCN Massachusetts data

In [33]:
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib as plc
from urllib.request import urlopen
import gzip
%matplotlib inline

### Read the data for Massachusetts from the USHCN Daily climate database

In [34]:
url="http://cdiac.ornl.gov/ftp/ushcn_daily/state19_MA.txt.gz"     #USHCN daily climate data for MA

f = urlopen(url)                                                  #get file object for URL contents
fg = gzip.open(f)                                                 #open with gzip to uncompress

fc = fg.readlines()                                               #get the lines from the uncompressed file

station_months = len(fc)                                          #each line is one station-month
print("Number of station months:{}".format(station_months))

Number of station months:51993


### Now figure out how many station days the data represents

In [35]:
ndays=[31,28,31,30,31,30,31,31,30,31,30,31]                       #number of days in each month

In [36]:
station_days=0
for i in range(0,station_months):
    coop_id  = fc[i][0:6].decode()                                #station id string
    yyyy     = fc[i][6:10].decode()                               #convert year from bytes to string 'yyyy'
    mm       = fc[i][10:12].decode()                              #convert month from bytes to string 'mm'
    year     = int(yyyy)                                          #get year as integer
    leapyear = ((year%4)==0 and (year%100)!=0) or (year%400)==0   #boolean: leapyear or not   
    month    = int(mm)                                            #get month as integer
    last_day = ndays[month-1]                                     #get the last day for this month from the table
    if (month==2) and leapyear:                                   #February has 29 days in a leap year
        last_day=29
    station_days = station_days + last_day                        #just count the station days

print("Total number of station days:{}".format(station_days))

Total number of station days:1582252


### Preallocate lists to allow reference by index

In [37]:
dd = {}                                         #allocate a dictionary for building the data frame
dd['coop_id'] = [None]*station_days             #populate a list for each column with 'None' for each line
dd['date']    = [None]*station_days             #  (this eliminates the need to do append() to add data)
dd['element'] = [None]*station_days
dd['value']   = [None]*station_days
dd['mflag']   = [None]*station_days
dd['qflag']   = [None]*station_days
dd['sflag']   = [None]*station_days
dd['header']  = [None]*station_days
dd['dstr']    = [None]*station_days

### Populate the lists from the data

In [38]:
day=0
for i in range(0,station_months):
    header   = fc[i][0:16].decode()                                #header for this record
    coop_id  = int(fc[i][0:6].decode())                            #station id string - convert to int
    yyyy     = fc[i][6:10].decode()                                #convert year from bytes to string 'yyyy'
    mm       = fc[i][10:12].decode()                               #convert month from bytes to string 'mm'
    element  = fc[i][12:16].decode()                               #element string
    
    year     = int(yyyy)                                           #get year as integer
    leapyear = ((year%4)==0 and (year%100)!=0) or (year%400)==0    #boolean: leapyear or not  
    month    = int(mm)                                             #get month as integer
    yyyymm   = yyyy + '-' + mm + '-'                               # 'yyyy-mm-' part of datestring
    
    last_day = ndays[month-1]                                      #get the last day for this month from the table
    if month==2 and leapyear:                                      #February has 29 days in a leap year
        last_day=29

    for d in range(1,last_day+1):                                  #loop through only as many days as there are this month
        offset             = 16+8*(d-1)                            #get the offset for this day
        try:
            dd['header'][day] = header                             #save the header
            dd['dstr'][day] = fc[i][offset,offset+8].decode()      #save the string for this day
            dd['coop_id'][day] = coop_id                           #update element list
            dd['element'][day] = element                           #update element list
            value = pd.to_numeric(fc[i][offset:offset+5])          #get 5-digit data value                         
            if value < -9990.0:                                    #change anything less than -9990 to NaN 
                value = np.nan
            dd['value'][day] = value                               #update values list
            dd['mflag'][day] = fc[i][offset+5:offset+6].decode()   #update mflag list - single character string
            dd['qflag'][day] = fc[i][offset+6:offset+7].decode()   #update qflag list - single character string
            dd['sflag'][day] = fc[i][offset+7:offset+8].decode()   #update sflag list - single character string
            daystr           ='{:02d}'.format(d)                   #convert day number to string 'dd' 
            try:
                date =pd.to_datetime(yyyymm + daystr)              #convert 'yyyy-mm-dd' to datetime value
            except ValueError:
                print("Invalid date: {}".format(yyyymm+daystr))
            dd['date'][day] = date                                 #update date list
        except IndexError:                                         #check for subscript out of bounds
            print("IndexError: days={}".format(days))
            print("i: {}".format(i))
        day=day+1                                                  #increment day number

### Use the populated lists to build a data frame

In [39]:
ushcn1 = pd.DataFrame(dd)                                          #construct DataFrame object from dictionary
ushcn1.head()                                                      #show the first few rows

,coop_id,date,element,line,mflag,qflag,sflag,value
0,190120,1893-01-01,TMAX,190120189301TMAX 44 6 55 6 31 6 21 ...,,,6,44.0
1,190120,1893-01-02,TMAX,190120189301TMAX 44 6 55 6 31 6 21 ...,,,6,55.0
2,190120,1893-01-03,TMAX,190120189301TMAX 44 6 55 6 31 6 21 ...,,,6,31.0
3,190120,1893-01-04,TMAX,190120189301TMAX 44 6 55 6 31 6 21 ...,,,6,21.0
4,190120,1893-01-05,TMAX,190120189301TMAX 44 6 55 6 31 6 21 ...,,,6,25.0


In [40]:
ushcn1.to_pickle('../ushcn1_MA.p')

### Combine rows so that there is one row per station per day, with all recorded data

In [47]:
#get precipitation data only - element 'PRCP' - and rename columns for merge
#
ushcn_PRCP = ushcn1[ushcn1['element']=='PRCP'].drop(['element'],axis=1)
ushcn_PRCP.head()
ushcn_PRCP.columns=['coop_id','date','mflag_PRCP', 'qflag_PRCP', 'sflag_PRCP','prcp']

#get minimum temperature data only - element 'TMIN' - and rename columns for merge
#
ushcn_TMIN = ushcn1[ushcn1['element']=='TMIN'].drop(['element','line'],axis=1)
ushcn_TMIN.columns=['coop_id','date','mflag_TMIN', 'qflag_TMIN', 'sflag_TMIN','tmin']

#get maximum temperature data only - element 'TMAX' - and rename columns for merge
#
ushcn_TMAX = ushcn1[ushcn1['element']=='TMAX'].drop(['element','line'],axis=1)
ushcn_TMAX.columns=['coop_id','date','mflag_TMAX', 'qflag_TMAX', 'sflag_TMAX','tmax']

#get snow data only - element 'SNOW' - and rename columns for merge
#
ushcn_SNOW = ushcn1[ushcn1['element']=='SNOW'].drop(['element','line'],axis=1)
ushcn_SNOW.columns=['coop_id','date','mflag_SNOW', 'qflag_SNOW', 'sflag_SNOW','snow']

#get snow depth data only - element 'SNWD' - and rename columns for merge
#
ushcn_SNWD = ushcn1[ushcn1['element']=='SNWD'].drop(['element','line'],axis=1)
ushcn_SNWD.columns=['coop_id','date','mflag_SNWD', 'qflag_SNWD', 'sflag_SNWD','snwd']

ValueError: Length mismatch: Expected axis has 7 elements, new values have 6 elements

### Merge by station id and date to create a single data frame

In [42]:
ushcna = ushcn_PRCP.merge(ushcn_SNOW,how='outer',on=['coop_id','date'])
ushcnb = ushcna.merge(ushcn_TMAX,how='outer',on=['coop_id','date'])
ushcnc = ushcnb.merge(ushcn_TMIN,how='outer',on=['coop_id','date'])
ushcnd = ushcnc.merge(ushcn_SNWD,how='outer',on=['coop_id','date'])
ushcnd.head()

,coop_id,date,mflag_PRCP,qflag_PRCP,sflag_PRCP,prcp,line,mflag_SNOW,qflag_SNOW,sflag_SNOW,...,sflag_TMAX,tmax,mflag_TMIN,qflag_TMIN,sflag_TMIN,tmin,mflag_SNWD,qflag_SNWD,sflag_SNWD,snwd
0,190120,1893-01-01,190120189301PRCP 0P 6 215 6 0P 6 0P...,P,,6,0.0,,,6,...,6,44.0,,,6,21.0,NaN,NaN,NaN,NaN
1,190120,1893-01-02,190120189301PRCP 0P 6 215 6 0P 6 0P...,,,6,215.0,,,6,...,6,55.0,,,6,32.0,NaN,NaN,NaN,NaN
2,190120,1893-01-03,190120189301PRCP 0P 6 215 6 0P 6 0P...,P,,6,0.0,,,6,...,6,31.0,,,6,10.0,NaN,NaN,NaN,NaN
3,190120,1893-01-04,190120189301PRCP 0P 6 215 6 0P 6 0P...,P,,6,0.0,,,6,...,6,21.0,,,6,4.0,NaN,NaN,NaN,NaN
4,190120,1893-01-05,190120189301PRCP 0P 6 215 6 0P 6 0P...,P,,6,0.0,,,6,...,6,25.0,,,6,15.0,NaN,NaN,NaN,NaN


### Get station data 

In [43]:
f =urlopen("http://cdiac.ornl.gov/ftp/ushcn_daily/ushcn-stations.txt")   #USHCN daily climate data stations file

coop_id   = []                                        #allocate lists to hold values
latitude  = []
longitude = []
state     = []
station   = []

for line in f:                                        #append values to lists
    coop_id.append(int(line[0:6].decode()))           #coop id
    latitude.append(pd.to_numeric(line[6:16]))        #latitude   
    longitude.append(pd.to_numeric(line[17:26]))      #latitude
    state.append(line[33:35].decode())                #state
    station.append(line[36:66].decode().strip())      #station name with trailing blanks stripped out

stations = pd.DataFrame(                              #construct DataFrame object from lists
    {'coop_id':coop_id,
     'latitude':latitude,
     'longitude':longitude,
     'state':state,
     'station':station
    })

f.close()

stations.head()

,coop_id,latitude,longitude,state,station
0,11084,31.0581,-87.0547,AL,BREWTON 3 SSE
1,12813,30.5467,-87.8808,AL,FAIRHOPE 2 NE
2,13160,32.8347,-88.1342,AL,GAINESVILLE LOCK
3,13511,32.7017,-87.5808,AL,GREENSBORO
4,13816,31.8700,-86.2542,AL,HIGHLAND HOME


In [44]:
stations[stations.state=='MA']

,coop_id,latitude,longitude,state,station
401,190120,42.3861,-72.5375,MA,AMHERST
402,190535,42.4833,-71.2833,MA,BEDFORD
403,190736,42.2122,-71.1136,MA,BLUE HILL
404,193213,42.1450,-73.4172,MA,GREAT BARRINGTON 5 SW
405,194105,42.6992,-71.1658,MA,LAWRENCE
406,195246,41.6333,-70.9333,MA,NEW BEDFORD
407,196486,41.9819,-70.6961,MA,PLYMOUTH-KINGSTON
408,196681,42.0500,-70.1833,MA,PROVINCETOWN
409,196783,42.5242,-71.1264,MA,READING
410,198367,41.9003,-71.0658,MA,TAUNTON


### Merge station data to produce final data frame

In [45]:
ushcn_MA = ushcnd.merge(stations,how='inner',on=['coop_id'])
ushcn_MA.shape

(351273, 27)

In [46]:
ushcn_MA.to_pickle('../ushcn_MA.p')